
# 포스터 트렌드 보고서 자동 생성기 — 텍스트 템플릿 (KOR)

**무엇을 하나요?**  
- 사용자가 `OO축제`를 입력하면, 유사한 축제 묶음을 추정하고  
- "잘 만들어진 포스터 Top5 형태"를 **텍스트**로 생성합니다.  
- 근거 문장에는 **네이버 검색량(공개 전/후)**, 선택적으로 **QR/UTM 반응**을 삽입할 자리가 있습니다.

> ⚠️ 이 노트북은 **템플릿 생성기**입니다. 네이버 데이터랩 API 호출은 포함하지 않고, 사용자가 가져온 수치를 대입합니다.


## 1) 입력 스키마

In [1]:

from dataclasses import dataclass, field
from typing import List, Optional, Dict

@dataclass
class SearchDelta:
    """네이버 데이터랩에서 가져온 전/후 검색량과 증가율(%)"""
    prev: int
    curr: int
    delta_pct: float  # (curr - prev)/prev*100

@dataclass
class PosterEvidence:
    """포스터 관련 근거값(선택값은 None 가능)"""
    search: Optional[SearchDelta] = None
    installs: Optional[int] = None
    qr_unique_scans: Optional[int] = None
    utm_sessions: Optional[int] = None
    conversions: Optional[int] = None

@dataclass
class TrendRequest:
    festival_name: str
    # 유사 축제 이름을 수동으로 넣을 수도 있고, 비우면 간단 키워드 규칙으로 제안
    similar_candidates: List[str] = field(default_factory=list)
    # 각 축제별 근거 수치(없으면 빈 dict로 두고 나중에 채워넣기)
    evidence_by_festival: Dict[str, PosterEvidence] = field(default_factory=dict)


## 2) 유사 축제 제안(간단 규칙)

In [2]:

import re
from typing import List

def guess_cluster(name: str) -> str:
    n = name.lower()
    if any(k in n for k in ["불꽃", "firework", "불꽃축제"]):
        return "fireworks"
    if any(k in n for k in ["빛", "초롱", "등불", "라이트", "light"]):
        return "light"
    if any(k in n for k in ["재즈", "jazz", "음악", "뮤직", "music"]):
        return "music"
    if any(k in n for k in ["영화", "영화제", "film", "biff"]):
        return "film"
    if any(k in n for k in ["산타", "크리스마스", "winter", "christmas"]):
        return "seasonal"
    return "general"

DEFAULT_SIMILARS = {
    "fireworks": ["서울세계불꽃축제", "부산불꽃축제", "포항불빛축제", "여수밤바다불꽃"],
    "light": ["서울빛초롱축제", "진주남강유등축제", "부산불빛축제", "광주빛고을"],
    "music": ["서울재즈페스티벌", "자라섬재즈페스티벌", "울산서머페스티벌", "펜타포트록페스티벌"],
    "film": ["부산국제영화제", "전주국제영화제", "부천국제판타스틱영화제", "제천국제음악영화제"],
    "seasonal": ["담양산타축제", "화천산천어축제", "홍천강꽁꽁축제", "태백산눈축제"],
    "general": ["지역문화축제A", "지역문화축제B", "도시브랜딩축제C"]
}

def suggest_similars(festival_name: str, provided: List[str]) -> List[str]:
    if provided:
        return provided
    cluster = guess_cluster(festival_name)
    return DEFAULT_SIMILARS.get(cluster, DEFAULT_SIMILARS["general"])[:4]


## 3) Top5 포스터 형태(클러스터별 추천)

In [3]:

from typing import Dict

POSTER_FORMS: Dict[str, list] = {
    "fireworks": [
        "야간 실사 사진 + 어두운 오버레이 + 굵은 일자/시간 블록",
        "하늘면 비우고 도시 스카이라인 하단 배치 + 대조 강한 타이포",
        "불꽃 일러스트 심벌 + 핵심정보 3요소(행사명·날짜·장소) 상단 정렬",
        "QR 우측하단 배치 + 예매/동선 아이콘 세트(3개 이하)",
        "색상 2~3색 제한 팔레트(딥블루/옐로/화이트) + 고대비"
    ],
    "light": [
        "글로우/네온 그라디언트 + 큰 타이포(세리프/산세리프 혼합 금지)",
        "야간 현장 사진 위 20% 어둡게 오버레이 + 좌측 상단 정보 박스",
        "랜턴/유등 심벌 반복 패턴 + 중앙 로고 띄움",
        "QR 하단 중앙 + 지도/시간 바로가기 칩",
        "색상은 보라/민트/옐로 중심의 고대비"
    ],
    "music": [
        "대담한 헤드라인(1~2단어) + 날짜·장소 칩 형태",
        "라인업 그리드(가로 3~4열) + 헤드라이너 대형",
        "사진/일러스트 혼합 금지, 하나만 택해 클린",
        "QR/단축URL을 라인업 하단 중앙 배치",
        "흑백 + 포인트컬러 1색(레드/옐로)"
    ],
    "film": [
        "심벌/엠블럼 중심(에디토리얼 레이아웃) + 여백 많이",
        "대비 높은 타이포(세리프 헤드, 산세리프 서브)",
        "상·하단 정보 라인 정렬, 중앙은 심벌 전체",
        "QR은 우측하단 미니 + '상영작/예매' 바로가기",
        "색상은 블랙/화이트/골드 등 고대비"
    ],
    "seasonal": [
        "마스코트(산타/눈) 일러스트 + 큰 날짜 로고타입",
        "패밀리 톤 사진 1장 + 상단 정보 리본",
        "QR은 하단 우측 + 이벤트/체험 아이콘 2~3개",
        "눈/별 패턴을 가장자리에만 배치(중앙은 깨끗하게)",
        "레드/그린/골드 2색만 사용해 난잡함 방지"
    ],
    "general": [
        "큰 타이포 + 고대비 배경(사진엔 20% 어둡게 오버레이)",
        "핵심 3요소(행사명·날짜·장소) 상단/중앙 3행",
        "QR/URL 명확히 + 행동유도 문구 5자 내외",
        "아이콘은 3개 이하, 로고는 하단 가로 정렬",
        "색상은 2~3색 제한, 과도한 그라디언트 금지"
    ]
}

def top5_forms_for(festival_name: str) -> list:
    cluster = guess_cluster(festival_name)
    return POSTER_FORMS.get(cluster, POSTER_FORMS["general"])[:5]


## 4) 문장 템플릿 생성

In [4]:

def make_block(req: TrendRequest) -> str:
    name = req.festival_name
    similars = suggest_similars(name, req.similar_candidates)
    forms = top5_forms_for(name)

    # 근거 문장 합성: 유사 축제들 중 검색량 근거가 있는 애만 사용
    evid_lines = []
    for sim in similars:
        ev = req.evidence_by_festival.get(sim)
        if ev and ev.search:
            evid_lines.append(
                f"「{sim}」의 포스터 공개 주간 검색량은 전주 대비 +{ev.search.delta_pct:.1f}%({ev.search.prev}→{ev.search.curr}) 변동을 보여, 포스터 공개가 관심 상승과 연동됨을 시사합니다."
            )
    evid_text = "\n".join(evid_lines) if evid_lines else "※ 유사 축제의 검색량 증감 데이터가 입력되지 않아 정량 근거는 생략합니다."

    forms_text = "\n".join([f"{i+1}) {f}" for i, f in enumerate(forms)])

    block = f"""-포스터 트랜드 분석-
최근 「{name}」와 유사한 축제들의 **잘 만들어진 포스터**는 다음과 같은 형태를 띱니다:

{forms_text}

그 근거로, 포스터 공개 전/후 **네이버 검색량**을 비교하면 관심의 변동을 수치로 확인할 수 있습니다.
{evid_text}

따라서 「{name}」에 권장되는 포스터 방향(Top5)은 위 항목과 같으며, 제작 시 반드시 **가독(큰 타이포·고대비)과 QR/단축URL 배치**를 포함해 오프라인→온라인 전환을 측정할 것을 권장합니다.
"""
    return block


## 5) 데모 실행(임의 수치 삽입 예시)

In [5]:

demo_req = TrendRequest(
    festival_name="담양산타축제",
    similar_candidates=[],  # 비워두면 자동 제안
    evidence_by_festival={
        "화천산천어축제": PosterEvidence(search=SearchDelta(prev=1150, curr=1480, delta_pct=(1480-1150)/1150*100)),
        "태백산눈축제": PosterEvidence(search=SearchDelta(prev=900, curr=1098, delta_pct=(1098-900)/900*100)),
    }
)

print(make_block(demo_req))


-포스터 트랜드 분석-
최근 「담양산타축제」와 유사한 축제들의 **잘 만들어진 포스터**는 다음과 같은 형태를 띱니다:

1) 마스코트(산타/눈) 일러스트 + 큰 날짜 로고타입
2) 패밀리 톤 사진 1장 + 상단 정보 리본
3) QR은 하단 우측 + 이벤트/체험 아이콘 2~3개
4) 눈/별 패턴을 가장자리에만 배치(중앙은 깨끗하게)
5) 레드/그린/골드 2색만 사용해 난잡함 방지

그 근거로, 포스터 공개 전/후 **네이버 검색량**을 비교하면 관심의 변동을 수치로 확인할 수 있습니다.
「화천산천어축제」의 포스터 공개 주간 검색량은 전주 대비 +28.7%(1150→1480) 변동을 보여, 포스터 공개가 관심 상승과 연동됨을 시사합니다.
「태백산눈축제」의 포스터 공개 주간 검색량은 전주 대비 +22.0%(900→1098) 변동을 보여, 포스터 공개가 관심 상승과 연동됨을 시사합니다.

따라서 「담양산타축제」에 권장되는 포스터 방향(Top5)은 위 항목과 같으며, 제작 시 반드시 **가독(큰 타이포·고대비)과 QR/단축URL 배치**를 포함해 오프라인→온라인 전환을 측정할 것을 권장합니다.

